In [1]:
# import tensorflow as tf
import numpy as np
import cv2
import os
from tqdm import tqdm

In [2]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split
import pandas as pd

C:\Users\Acer\AppData\Local\Temp\ipykernel_13084\3900097976.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
from os import listdir
from os.path import isfile, join

In [4]:
# not sure the real size
width = 450
hieght = 550

In [11]:
df = pd.read_excel(r"data\cdi-data-scan1266.xlsx", sheet_name="depressed-data")
# df = df[["ID","Pt","DEP","Age","Sex"]] #1 = depressed

In [ ]:
keys = []
keys.append("Age")
keys.append("Sex")
keys

In [13]:
def labelfeatures(ID):
    i = df['Pt'][df['ID']==ID].index
    # score = df.loc[0, "Pt"]
    score = df.loc[i[0], keys]
    score.to_numpy()
    return score

In [12]:
def labelY(ID):
    i = df['Pt'][df['ID']==ID].index
    # score = df.loc[0, "Pt"]
    y = df.loc[i[0], "Pt"]
    return y

In [14]:
def img2data(path):
  img_data = []
  IDs = []
  # features = []
  Y = []

  for folder in os.listdir(path):
    folder_path = os.path.join(path, folder)
    for item in tqdm(os.listdir(folder_path)):
      filePath = os.path.join(folder_path, item)

      # print(item)
      ID = int(item.split('.')[0])
      IDs.append(ID)
      # features.append(labelfeatures(ID))
      Y.append(labelY(ID))

      img = cv2.imread(filePath, cv2.IMREAD_GRAYSCALE)
      # plt.imshow(img)

      img = cv2.resize(img , (width, hieght))
      # img.flatten()
      # img.append(score)
      # img = cv2.mean(img)

      img_data.append(img)

  return img_data, IDs, Y

In [15]:
img, img_name, Y = img2data(r"data_label")
# print(img)

img = np.asarray(img)

flattened_images = img.reshape(img.shape[0], -1)

  0%|          | 0/204 [00:00<?, ?it/s]

100%|██████████| 266/266 [00:07<00:00, 33.38it/s]


In [ ]:
# analyze with score feature
# images_score = np.column_stack((img, features))

In [17]:
# analyze with images only
images_score = flattened_images
images_score.shape

(470, 247500)

In [ ]:
k = 2

In [ ]:
kmeans = KMeans(k, random_state = 40)
kmeans.fit(images_score)

In [21]:
birch = Birch(threshold=0.5, n_clusters=k)
birch.fit(images_score)

In [ ]:
gaussian = GaussianMixture(n_components=k, random_state=40)
gaussian.fit(images_score)

In [ ]:
amt = AgglomerativeClustering(n_clusters=k)
amt.fit(images_score)

In [ ]:
image_cluster = pd.DataFrame(img_name, columns=['image'])

In [ ]:
def labelscore(ID):
    
    i = df['Pt'][df['ID']==ID].index
    score = df.loc[i[0], 'Pt']
    return score

In [ ]:
image_cluster["BIRCH-cluster"] = birch.labels_
image_cluster["KMeans-cluster"] = kmeans.labels_
image_cluster["Gaussian-cluster"] = gaussian.predict(images_score)
image_cluster["Agglomerative-cluster"] = amt.labels_
image_cluster["score"] = Y
image_cluster

In [ ]:
def labelClass(ID):
    
    i = df['DEP'][df['ID']==ID].index
    score = df.loc[i[0], 'DEP']
    return score

def clusterClass(ID):
    i = image_cluster['clusterid'][image_cluster['image']==ID].index
    # print(i)
    score = image_cluster.loc[i[0], 'clusterid']
    return score

In [ ]:
y_true = []

for id in image_cluster['image']:
    y_true.append(labelClass(id))

In [ ]:
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [ ]:

for Ypred in images_score[-5:-1].columns:
    print(Ypred)
    print(f"adjusted_mutual_info_score : {adjusted_mutual_info_score(y_true, image_cluster[Ypred])}")
    print(f"F1-score : {f1_score(y_true, image_cluster[Ypred])}")
    print(f"accuracy_score of : {accuracy_score(y_true, image_cluster['cluster'])}")
    print()

In [ ]:
for Ypred in images_score[-5:-1].columns:
    cm = confusion_matrix(y_true, image_cluster[Ypred])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.show()

In [ ]:
# name_cluster = (
#     "Cluster 0",
#     "Cluster 1",
# )
# weight_counts = {
#     "dep": cm[0],
#     "no-dep": cm[1],
# }
# width = 0.5

# fig, ax = plt.subplots()
# bottom = np.zeros(2)

# for boolean, weight_count in weight_counts.items():
#     p = ax.bar(name_cluster, weight_count, width, label=boolean, bottom=bottom)
#     bottom += weight_count

# ax.set_title("Number of depressed in each cluster")
# ax.legend(loc="upper right")

# plt.show()

In [ ]:
# centroids = kmeans.cluster_centers_
# centroids

In [ ]:
# plt.scatter(image_cluster["score"], images_score[:, 0], c= image_cluster["cluster"], cmap='viridis')
# plt.scatter( centroids[:, 1], centroids[:, 0], s=200, c='red', marker='x')
# plt.title('K-Means Clustering')
# plt.xlabel('color mean')
# plt.ylabel('CDI score')
# plt.show()

In [ ]:
# import seaborn as sn

In [ ]:
#visulaize

# fig = plt.figure(figsize=(300, 100))
# rows = 15
# columns = 40 
# i = 0
# for ID in image_cluster["image"][image_cluster['cluster']==1][244:]:
#     i += 1
#     name = str(ID).rjust(3, '0')
#     folname = labelClass(ID)

#     if folname == 0:
#         folname = "Depressed"
#     else:
#         folname = "No-Depressed"
#     # 0 = dep
#     # 1 = no-dep

#     image = cv2.imread(f'data_label\\{folname}\\{name}.jpg') 
#     fig.add_subplot(rows, columns, i) 

#     plt.imshow(image) 
#     plt.axis('off') 